In [1]:
import cirq
from cirq_iqm import Adonis, XYGate, IsingGate

# Adonis

Qubit connectivity:
```
      QB1
       |
QB4 - QB3 - QB2
       |
      QB5
```

In [2]:
adonis = Adonis()
print(adonis.NATIVE_GATES)
print(adonis.qubits)

(<class 'cirq.ops.phased_x_gate.PhasedXPowGate'>, <class 'cirq.ops.common_gates.XPowGate'>, <class 'cirq.ops.common_gates.YPowGate'>, <class 'cirq.ops.common_gates.ZPowGate'>, <class 'cirq_iqm.iqm_gates.IsingGate'>, <class 'cirq_iqm.iqm_gates.XYGate'>, <class 'cirq.ops.measurement_gate.MeasurementGate'>)
(IQMQubit(1), IQMQubit(2), IQMQubit(3), IQMQubit(4), IQMQubit(5))


-----
Create a quantum circuit and insert native gates

In [3]:
QB1, QB2, QB3 = adonis.qubits[:3]
native_circuit = cirq.Circuit(device=adonis)
native_circuit.append(cirq.X(QB1))
native_circuit.append(XYGate(exponent=0.5)(QB1, QB3))
native_circuit.append(cirq.PhasedXPowGate(phase_exponent=0.3, exponent=0.5)(QB3))
native_circuit.append(IsingGate(exponent=0.2)(QB1, QB3))
native_circuit.append(cirq.measure(QB3, key='QB3_result'))
print(native_circuit)

QB1: ───X───XY──────────────────────ZZ─────────────────────────
            │                       │
QB3: ───────XY^0.5───PhX(0.3)^0.5───ZZ^0.2───M('QB3_result')───


-----
Insert non-native gates, which are immediately decomposed to native ones

In [4]:
arb_circuit = cirq.Circuit(device=adonis)
arb_circuit.append(cirq.H(QB1))
arb_circuit.append(cirq.CNOT(QB1, QB3))
arb_circuit.append(cirq.measure(QB3, key='QB3_result'))
print(arb_circuit)

QB1: ───Y^0.5────X───ZZ───────S^-1─────────────────────────────
                     │
QB3: ───Y^-0.5───────ZZ^0.5───S^-1───Y^0.5───M('QB3_result')───


-----
Optimize the circuit

In [5]:
adonis.simplify_circuit(arb_circuit)
print(arb_circuit)

QB1: ───PhX(-0.5)^0.5───ZZ───────S──────────────────────────────
                        │
QB3: ───PhX(-0.5)^0.5───ZZ^0.5───PhX(1)^0.5───M('QB3_result')───


-----
See `examples/qasm_demo.py` for more examples
